### Project - Auto Theft 

In [1]:
import pandas as pd
import numpy as np

#load data 

# clean auto_theft data 
auto_theft = pd.read_csv('Auto_Theft_Open_Data.csv')

auto_theft = auto_theft.rename(columns = {'X':'Longitude', 'Y':'Latitude', 'NEIGHBOURHOOD_158':'Neighbourhood'})
auto_theft.columns = map(str.lower, auto_theft.columns)

auto_theft = auto_theft.drop(columns = ['long_wgs84', 'lat_wgs84'])

print(auto_theft.isna().any())

#clean demographics data 

demo = pd.read_excel('wellbeing-toronto-demographics.xlsx', header = 1, index_col = 'Neighbourhood')
demo.columns = map(str.lower, demo.columns)

demo = demo.drop(columns = ['neighbourhood id'])

# clean economics data

econ = pd.read_excel('wellbeing-toronto-economics.xlsx', header = 1, index_col = 'Neighbourhood')
econ.columns = map(str.lower, econ.columns)

econ = econ.drop(columns = ['neighbourhood id'])

#clean housing data

housing = pd.read_excel('wellbeing-toronto-housing.xlsx', index_col = 'Neighbourhood')

housing = housing.drop(columns = ['Neighbourhood Id'])

FileNotFoundError: [Errno 2] No such file or directory: 'Auto_Theft_Open_Data.csv'

In [3]:
# join tables econ, demo, housing for cluster analysis 

neighbourhood_join = demo.join(econ, on = "Neighbourhood", how = 'left', lsuffix = 'demo', rsuffix = 'econ')

neighbourhood_join = neighbourhood_join.join(housing, on = "Neighbourhood", how = 'left', lsuffix = 'neighbourhood_join', rsuffix = 'housing')

neighbourhood_join.head(1)


,total area,total population,pop - males,pop - females,pop 0 - 4 years,pop 5 - 9 years,pop 10 - 14 years,pop 15 -19 years,pop 20 - 24 years,pop 25 - 29 years,...,Home Prices,Mid-Century Highrise Households,Mid-Century Highrise Population,Percent Mid-Century Highrise Households,Percent Mid-Century Highrise Population,Rent Bank Applicants,Social Housing Turnover,Social Housing Units,Social Housing Waiting List,Cluster Label
Neighbourhood,,,,,,,,,,,,,,,,,,,,,
West Humber-Clairville,30.09,34100,17095,17000,1865,1950,2155,2550,2855,2755,...,317508,690,1810,6.8,5.3,38,0.312500,952,557,5
Mount Olive-Silverstone-Jamestown,4.60,32790,16015,16765,2575,2535,2555,2620,2400,2335,...,251119,4110,13395,42.9,40.9,41,7.000000,1146,1049,5
Thistletown-Beaumond Heights,3.40,10140,4920,5225,575,580,670,675,675,675,...,414216,430,1260,13.4,12.4,7,1.333333,447,270,2
Rexdale-Kipling,2.50,10485,5035,5455,495,520,570,665,650,645,...,392271,600,1050,15.2,10.0,19,4.200000,432,186,2
Elms-Old Rexdale,2.90,9550,4615,4935,670,720,720,725,655,570,...,233832,870,2305,27.3,24.1,14,1.000000,463,239,2


In [4]:
# cluster analysis 

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import random
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering
from sklearn import preprocessing
random.seed(42)

# normalize data 
for column in neighbourhood_join.columns:
    neighbourhood_join[column] = (neighbourhood_join[column] - neighbourhood_join[column].min()) / (neighbourhood_join[column].max() - neighbourhood_join[column].min())  

    

In [18]:
# apply kmeans algorithm to data with k=6

X = neighbourhood_join

kmeans = KMeans(n_clusters=6, random_state=0).fit(X) #fit model
cluster_label = kmeans.labels_

label = kmeans.fit_predict(X)

demo['Cluster Label'] = label


In [20]:
# join tables by neibourhood column for machine learning 

auto_theft_join = auto_theft.join(demo, on = "neighbourhood")

auto_theft_join = auto_theft_join.join(econ, on = "neighbourhood", how = 'left', lsuffix = 'auto_theft_join', rsuffix = 'econ')

auto_theft_join = auto_theft_join.join(housing, on = "neighbourhood", how = 'left', lsuffix = 'auto_theft_join', rsuffix = 'housing')

auto_theft_join.columns

auto_theft_join.head()


,longitude,latitude,objectid,event_unique_id,report_date,occ_date,report_year,report_month,report_day,report_doy,...,social assistance recipients,Home Prices,Mid-Century Highrise Households,Mid-Century Highrise Population,Percent Mid-Century Highrise Households,Percent Mid-Century Highrise Population,Rent Bank Applicants,Social Housing Turnover,Social Housing Units,Social Housing Waiting List
0,-79.306757,43.734652,1,GO-20141263217,2014/01/01 05:00:00+00,2013/12/31 05:00:00+00,2014,January,1,1,...,2593.0,365107.0,2345.0,5410.0,32.2,31.5,15.0,7.083333,891.0,459.0
1,-79.236189,43.721847,2,GO-20141262914,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014,January,1,1,...,1342.0,542218.0,1100.0,2080.0,18.9,13.2,10.0,5.875000,497.0,289.0
2,-79.529478,43.618472,3,GO-20141262837,2014/01/01 05:00:00+00,2013/12/25 05:00:00+00,2014,January,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-79.516814,43.683438,4,GO-20141265795,2014/01/02 05:00:00+00,2014/01/01 05:00:00+00,2014,January,2,2,...,751.0,873268.0,1550.0,3630.0,25.7,24.3,10.0,0.000000,0.0,139.0
4,-79.595364,43.744281,5,GO-20141265947,2014/01/02 05:00:00+00,2014/01/01 05:00:00+00,2014,January,2,2,...,6561.0,251119.0,4110.0,13395.0,42.9,40.9,41.0,7.000000,1146.0,1049.0


In [ ]:
# logistic regression

